In [1]:
import pm4py
import sys
sys.path.append('../../TaskExecutionTimeMining/')
from event_log_transformer import *
pandas.set_option('display.max_columns', None)

In [2]:
file_path = '../../../data/BPI Challenge 2017.xes'
event_log = pm4py.read_xes(file_path)

parsing log, completed traces ::   0%|          | 0/31509 [00:00<?, ?it/s]

In [3]:
event_log

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID
0,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 09:51:15.304000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,2016-01-01 09:51:15.352000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,2016-01-01 09:51:15.774000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202262,Deleted,User_1,W_Call after offers,Workflow,Workitem_1817549786,ate_abort,2017-01-06 06:33:02.212000+00:00,Home improvement,New credit,Application_1350494635,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1202263,Created,User_1,W_Call after offers,Workflow,Workitem_363876066,schedule,2017-01-06 06:33:02.221000+00:00,Home improvement,New credit,Application_1350494635,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1202264,statechange,User_28,A_Cancelled,Application,ApplState_1869071797,complete,2017-01-16 09:51:21.114000+00:00,Home improvement,New credit,Application_1350494635,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1202265,statechange,User_28,O_Cancelled,Offer,OfferState_420066181,complete,2017-01-16 09:51:21.139000+00:00,Home improvement,New credit,Application_1350494635,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Offer_1580299144


In [4]:
event_log['concept'] = event_log['concept:name'] + '_' + event_log['lifecycle:transition']
event_log = event_log[event_log['concept:name'].str.startswith('W_')]

In [5]:
event_log[:99999].to_csv('test.csv', index=False, date_format='%Y-%m-%d %H:%M:%S.%f')

In [6]:
event_log['lifecycle:transition'].unique()

array(['schedule', 'withdraw', 'start', 'suspend', 'ate_abort', 'resume',
       'complete'], dtype=object)

In [7]:
start_end_event_log = TransformEventLog.start_end_event_log_mult(event_log,
                                                                   start_name_1 = 'start', start_name_2 = 'resume',
                                                                   complete_name_3 = 'suspend',
                                                                   complete_name_1 = 'complete', complete_name_2 = 'ate_abort')

/home/michel/Documents/TaskExecutionTimeMining/src/notebooks/BPIC_2017/../../TaskExecutionTimeMining/event_log_transformer.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_end_event_log.loc[:, 'duration'] = start_end_event_log[timestamp_name + '_complete'] - start_end_event_log[timestamp_name + '_start']
/home/michel/Documents/TaskExecutionTimeMining/src/notebooks/BPIC_2017/../../TaskExecutionTimeMining/event_log_transformer.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_end_event_log.

In [8]:
# Seconds in day
start_end_event_log = TransformEventLog.seconds_in_day(start_end_event_log, 'time:timestamp_start')

In [9]:
start_end_event_log

,Action_start,org:resource,concept:name,EventOrigin_start,EventID_start,lifecycle:transition_start,time:timestamp_start,case:LoanGoal_start,case:ApplicationType_start,case:concept:name,case:RequestedAmount_start,FirstWithdrawalAmount_start,NumberOfTerms_start,Accepted_start,MonthlyCost_start,Selected_start,CreditScore_start,OfferedAmount_start,OfferID_start,concept_start,Action_complete,EventOrigin_complete,EventID_complete,lifecycle:transition_complete,time:timestamp_complete,case:LoanGoal_complete,case:ApplicationType_complete,case:RequestedAmount_complete,FirstWithdrawalAmount_complete,NumberOfTerms_complete,Accepted_complete,MonthlyCost_complete,Selected_complete,CreditScore_complete,OfferedAmount_complete,OfferID_complete,concept_complete,duration,duration_seconds,seconds_in_day
1896927,Obtained,User_80,W_Call after offers,Workflow,Workitem_1000013868,start,2016-10-18 08:54:52.604000+00:00,Car,New credit,Application_1804686886,6500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_Call after offers_start,Released,Workflow,Workitem_282145227,suspend,2016-10-18 08:57:35.834000+00:00,Car,New credit,6500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_Call after offers_suspend,0 days 00:02:43.230000,163,32092
72020,Obtained,User_114,W_Validate application,Workflow,Workitem_1000015916,start,2016-01-19 12:58:12.261000+00:00,Car,New credit,Application_704665572,15000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_Validate application_start,Released,Workflow,Workitem_363314289,suspend,2016-01-19 13:01:59.549000+00:00,Car,New credit,15000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_Validate application_suspend,0 days 00:03:47.288000,227,46692
2149692,Obtained,User_51,W_Call incomplete files,Workflow,Workitem_1000016777,resume,2016-12-01 19:01:26.846000+00:00,Home improvement,New credit,Application_907188218,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_Call incomplete files_resume,Released,Workflow,Workitem_679245376,suspend,2016-12-01 19:04:53.209000+00:00,Home improvement,New credit,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_Call incomplete files_suspend,0 days 00:03:26.363000,206,68486
236762,Obtained,User_5,W_Call after offers,Workflow,Workitem_1000019350,resume,2016-02-13 15:34:25.947000+00:00,Home improvement,New credit,Application_2110398373,8500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_Call after offers_resume,Released,Workflow,Workitem_365997120,suspend,2016-02-13 15:34:35.927000+00:00,Home improvement,New credit,8500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_Call after offers_suspend,0 days 00:00:09.980000,9,56065
1027441,Obtained,User_61,W_Call incomplete files,Workflow,Workitem_1000021441,resume,2016-06-20 14:00:07.911000+00:00,Home improvement,New credit,Application_463742835,40000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_Call incomplete files_resume,Released,Workflow,Workitem_835600192,suspend,2016-06-20 14:02:24.019000+00:00,Home improvement,New credit,40000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_Call incomplete files_suspend,0 days 00:02:16.108000,136,50407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714004,Obtained,User_112,W_Validate application,Workflow,Workitem_99990472,start,2016-05-02 10:19:49.618000+00:00,Not speficied,New credit,Application_422800640,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_Validate application_start,Released,Workflow,Workitem_1738241612,suspend,2016-05-02 10:21:22.923000+00:00,Not speficied,New credit,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_Validate application_suspend,0 days 00:01:33.305000,93,37189
1837433,Obtained,User_131,W_Validate application,Workflow,Workitem_999916513,start,2016-10-18 07:50:28.148000+00:00,Home improvement,New credit,Application_480209137,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_Validate application_start,Released,Workflow,Workitem_616491584,suspend,2016-10-18 07:50:41.170000+00:00,Home improvement,New credit,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W_Validate application_suspend,0 days 00:00:13.022000,13,28228
19

In [12]:
gb = start_end_event_log.groupby(['case:concept:name', 'org:resource', 'concept:name']).count()['Action_start'].to_frame()

In [13]:
gb[gb['Action_start'] > 8]

Action_start
case:concept:name      org:resource concept:name                          
Application_1056033042 User_83      W_Validate application              14
Application_1106557768 User_99      W_Call incomplete files              9
Application_1141800606 User_100     W_Assess potential fraud            13
Application_1153198699 User_93      W_Validate application               9
Application_1186028058 User_83      W_Validate application              10
...                                                                    ...
Application_949646433  User_46      W_Complete application               9
Application_984284284  User_29      W_Validate application               9
                       User_95      W_Validate application               9
Application_994471794  User_87      W_Validate application               9
Application_995124359  User_68      W_Validate application               9

[68 rows x 1 columns]

In [15]:
start_end_event_log[(start_end_event_log['case:concept:name'] == 'Application_949646433')].sort_values('time:timestamp_start')\
[['case:concept:name', 'org:resource', 'concept:name', 'lifecycle:transition_start', 'time:timestamp_start', 'lifecycle:transition_complete', 'time:timestamp_complete', 'duration_seconds']]

,case:concept:name,org:resource,concept:name,lifecycle:transition_start,time:timestamp_start,lifecycle:transition_complete,time:timestamp_complete,duration_seconds
1309569,Application_949646433,User_46,W_Handle leads,start,2016-07-16 14:46:30.839000+00:00,complete,2016-07-16 14:49:23.878000+00:00,173
1309593,Application_949646433,User_46,W_Complete application,start,2016-07-16 14:49:23.887000+00:00,suspend,2016-07-16 14:49:57.284000+00:00,33
1309630,Application_949646433,User_53,W_Complete application,resume,2016-07-18 07:11:08.223000+00:00,suspend,2016-07-18 07:12:17.434000+00:00,69
1309637,Application_949646433,User_46,W_Complete application,resume,2016-07-18 07:17:18.340000+00:00,suspend,2016-07-18 07:18:35.788000+00:00,77
1309677,Application_949646433,User_46,W_Complete application,resume,2016-07-18 07:44:34.774000+00:00,suspend,2016-07-18 07:45:21.420000+00:00,46
1309710,Application_949646433,User_54,W_Complete application,resume,2016-07-18 10:03:28.976000+00:00,suspend,2016-07-18 10:04:12.693000+00:00,43
1309720,Application_949646433,User_54,W_Complete application,resume,2016-07-18 10:09:56.230000+00:00,suspend,2016-07-18 10:10:15.073000+00:00,18
1309733,Application_949646433,User_46,W_Complete application,resume,2016-07-18 10:27:52.833000+00:00,suspend,2016-07-18 10:28:10.204000+00:00,17
1309773,Application_949646433,User_46,W_Complete application,resume,2016-07-18 10:31:03.395000+00:00,suspend,2016-07-18 10:31:10.893000+00:00,7
1309813,Application_949646433,User_46,W_Complete application,resume,2016-07-18 10:40:27.165000+00:00,suspend,2016-07-18 10:40:38.881000+00:00,11


In [16]:
# Resource counts

resource_count_event_log = TransformEventLog.value_count_per_case(start_end_event_log, 'org:resource',
                                                                  timestamp_name = 'time:timestamp_start',
                                                                 lifecycle_col_name = 'lifecycle:transition_start')

/home/michel/Documents/TaskExecutionTimeMining/src/notebooks/BPIC_2017/../../TaskExecutionTimeMining/event_log_transformer.py:138: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  value_count_event_log = value_count_event_log.fillna(0)


In [ ]:
resource_count_event_log

In [ ]:
activity_count_event_log = TransformEventLog.value_count_per_case(resource_count_event_log, 'concept:name',
                                                                  timestamp_name = 'time:timestamp_start',
                                                                 lifecycle_col_name = 'lifecycle:transition_start' )

In [ ]:
activity_count_event_log

In [ ]:
activity_count_event_log.to_csv('../transformed_event_logs/BPIC_2017_start_end.csv')
activity_count_event_log.to_pickle('../transformed_event_logs/BPIC_2017_start_end.pickle')